# Ordenação pelo Particionamento usando a Raiz Quadráda

## Resumo


### Pacotes instalados

In [3]:
using Random

In [2]:
# using Pkg

In [3]:
# Pkg.add("SortingAlgorithms")

In [4]:
using SortingAlgorithms

### Função Particionar

In [5]:
function particionar(enesima_particao::Int64, tamanho_particao::Int64, tamanho_vetor::Int64, vetor::Vector{Int64}, vetor_particionado::Vector{Vector{Int64}})
    k::Int64 = 0
    j::Int64 = 0

    @simd for k = tamanho_particao:tamanho_particao:tamanho_vetor # c n^0.5
        @inbounds vetor[j+1:k] = sort(vetor[j + 1:k], alg=InsertionSort) # c sum n^0.5 até n + n
        @inbounds push!(vetor_particionado, vetor[j+1:k]) # c sum n^0.5 até n + n
        if (enesima_particao != 0 && k == tamanho_vetor - enesima_particao)
            @inbounds vetor[k+enesima_particao:k+enesima_particao] = sort(vetor[k + enesima_particao:k + enesima_particao], alg=InsertionSort) #c n mod n^0.5 + c
            @inbounds push!(vetor_particionado, vetor[k+enesima_particao:k+enesima_particao]) # c n mod n^0.5
        end
        j = k # c n^{0.5}
    end
    return vetor_particionado
end

particionar (generic function with 1 method)

### Função Juntar

In [6]:
function juntar(tamanho_vetor::Int64, vetor_solucao::Vector{Int64}, vetor_particionado::Vector{Vector{Int64}})
    maximo::Int64 = 0
    indice_conjunto::Int64 = 0
    indice_particao::Int64 = 0
    @simd for _ = 1:tamanho_vetor # c n
        @simd for k in vetor_particionado # c n^0.5 de 1 até n
            if !isempty(k)
                @inbounds if maximo < k[length(k)] # c n^0.5 - 1
                    @inbounds maximo = k[length(k)] #c 1
                    @inbounds indice_conjunto = findall(x->x==k, vetor_particionado)[1] # c n^0.5 - 1
                    indice_particao = length(k) # c sum n^{5}
                end
            end
        end
        
        @inbounds pushfirst!(vetor_solucao, maximo) # c n - 1
        if !isempty(vetor_particionado[indice_conjunto])
            @inbounds deleteat!(vetor_particionado[indice_conjunto], indice_particao) # c n - 1
        end
        maximo = -2^(63)
    end
    return vetor_solucao
end

juntar (generic function with 1 method)

### Main

In [7]:
function sqrtSort(vetor::Vector{Int64}) where {Int64}
    tamanho_vetor::Int64 = 0
    enesima_particao::Int64 = 0
    tamanho_particao::Int64 = 0
    vetor_solucao::Vector{Int64} = Vector{Int64}(undef,0)
    vetor_intervalo_particionado::Vector{Int64} = Vector{Int64}(undef,0)
    vetor_particionado::Vector{Vector{Int64}} = Vector{Vector{Int64}}(undef, 0)

    tamanho_vetor = length(vetor)
    tamanho_particao = Int(trunc(sqrt(tamanho_vetor)))
    enesima_particao = rem(tamanho_vetor, tamanho_particao)
    
        
    vetor_particionado = particionar(
            enesima_particao,
            tamanho_particao,
            tamanho_vetor, 
            vetor,
            vetor_particionado
    )
    
    juntar(
        tamanho_vetor,
        vetor_solucao,
        vetor_particionado
    )
    
    return vetor_solucao
end

sqrtSort (generic function with 1 method)

### Teste

In [15]:
function f1()
  vetor::Vector{Int64} = rand(Int64, 10)
  @time vetor_solucao = sqrtSort(vetor)
end

f1 (generic function with 1 method)

In [9]:
function f2()
    vetor::Vector{Int64} = rand(Int64, 100)
    @time vetor_solucao = sqrtSort(vetor)
end

f2 (generic function with 1 method)

In [8]:
function f3()
  vetor::Vector{Int64} = rand(Int64, 1000)
  @time vetor_solucao = sqrtSort(vetor)
end

f3 (generic function with 1 method)

In [18]:
function f4()
  vetor::Vector{Int64} = rand(Int64, 10000)
  @time vetor_solucao = sqrtSort(vetor)
end

f4 (generic function with 1 method)

In [19]:
function f5()
  vetor::Vector{Int64} = rand(Int64, 100000)
  @time vetor_solucao = sqrtSort(vetor)
end

f5 (generic function with 1 method)

In [20]:
function f6()
  vetor::Vector{Int64} = rand(Int64, 1000000)
  @time vetor_solucao = sqrtSort(vetor)
end

f6 (generic function with 1 method)

In [21]:
function f7()
  vetor::Vector{Int64} = rand(Int64, 10000000)
  @time vetor_solucao = sqrtSort(vetor)
end

f7 (generic function with 1 method)

In [22]:
@time f1()

  0.218443 seconds (302.02 k allocations: 16.219 MiB, 99.96% compilation time)
  0.309267 seconds (320.80 k allocations: 17.451 MiB, 93.60% compilation time)


10-element Vector{Int64}:
 -6713772302423863707
 -5591902783780751559
 -3481251736528237610
 -1369671260792274562
  -517802285714479169
   -39269830224170673
  4185723834833254104
  4793560041261470790
  4869153343192072547
  8262100160120403640

In [10]:
f2()

  0.260561 seconds (303.56 k allocations: 16.278 MiB, 99.88% compilation time)


100-element Vector{Int64}:
 -9061168867511378882
 -8535651507949771165
 -8274034201656134799
 -7458157698177702161
 -7243942881598378026
 -6993626960438031252
 -6819479968765817548
 -6685197464413176767
 -6622624993348434681
 -6434660065409238520
 -6415636939904253348
 -6312132328520459339
 -6158843549871242587
                    ⋮
  6419217689500333146
  6654643366754878295
  7107981027314989021
  7205646250239752322
  7243539384983362567
  7701789592882647458
  8011791970283865044
  8361121772557006321
  8382366571183583022
  8385329146415252414
  8685323857838249428
  8815290518692398219

In [23]:
f3()

  0.004885 seconds (23.91 k allocations: 918.844 KiB)


1000-element Vector{Int64}:
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9221052099668049641
 -9219079659070369917
 -9216702829448661117
 -9199849087014855523
 -9185108097038026886
 -9155325745915312240
                    ⋮
  9086296418997570395
  9086814171766291310
  9091080488762948246
  9094747212151842068
  9106161517003809199
  9109430833744062413
  9122078800394552200
  9132272160815432716
  9156664370644565316
  9167289585697661973
  9177557443700835383
  9199572410923210953

In [24]:
f4()

  0.125436 seconds (311.37 k allocations: 12.443 MiB)


10000-element Vector{Int64}:
 -9221323576812053298
 -9219210930395581075
 -9217260419618908874
 -9211659664442516504
 -9210940650570060788
 -9210834736452737314
 -9208853734872259820
 -9206186955415475117
 -9204352587495643227
 -9201432916043492308
 -9198269755681988231
 -9195122813135082231
 -9193900756304246106
                    ⋮
  9205189456925854541
  9206172583919067202
  9207012172768173061
  9207370084543135284
  9210833089380953098
  9213117088083891316
  9213368396939869226
  9215617977802037508
  9218235883186932290
  9221456591809019943
  9222385761494355507
  9223325709378456887

In [25]:
f5()

  2.993587 seconds (4.46 M allocations: 2.698 GiB, 7.46% gc time)


100000-element Vector{Int64}:
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
                    ⋮
  9221264811728685288
  9221273661924057117
  9221610308619691075
  9221927346183136356
  9222018064072200070
  9222152316658333916
  9222164129128855620
  9222493312280399680
  9222738183844632888
  9222942491780711257
  9223034055767561192
  9223261679250598406

In [26]:
f6()

 68.228278 seconds (52.44 M allocations: 32.396 GiB, 5.59% gc time)


1000000-element Vector{Int64}:
 -9223354835006886285
 -9223352914581862355
 -9223299499838221273
 -9223287803800678138
 -9223283751240312496
 -9223277260497213359
 -9223272419810748986
 -9223267575272079309
 -9223250108280368566
 -9223247058941191526
 -9223242227034408355
 -9223222642345887068
 -9223218974710824487
                    ⋮
  9223126300354011039
  9223129409872019068
  9223134803360430247
  9223151851226834944
  9223157870742415163
  9223221974374166044
  9223237246355701452
  9223253912612535371
  9223299618049286641
  9223307034365755267
  9223350614464363218
  9223359619850944767

In [27]:
f7()

1179.790976 seconds (604.25 M allocations: 397.675 GiB, 1.56% gc time)


10000000-element Vector{Int64}:
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
 -9223372036854775808
                    ⋮
  9223347508153394355
  9223348588792782941
  9223355407136468143
  9223357495608388580
  9223358493299237594
  9223360752193881463
  9223362130325981774
  9223365686396373139
  9223367796601588683
  9223367916340502416
  9223368217250267005
  9223368726089216683